JupyterHub has a [REST API](http://petstore.swagger.io/?url=https://raw.githubusercontent.com/jupyterhub/jupyterhub/master/docs/rest-api.yml#!/default/get_users):

In [1]:
import requests
hub_api = 'http://127.0.0.1:8081/hub/api/'

token = "result of token=$(jupyterhub token yourname)"

In [2]:
import json
from requests import HTTPError

def api_request(path, method='get', data=None):
    if data:
        data = json.dumps(data)
    
    r = requests.request(method, hub_api + path,
        headers={'Authorization': 'token %s' % token},
        data=data,
    )
    try:
        r.raise_for_status()
    except Exception as e:
        try:
            info = r.json()
        except Exception:
            raise e
        if 'message' in info:
            # raise nice json error if there was one
            raise HTTPError("%s: %s" % (r.status_code, info['message'])) from None
        else:
            # raise original
            raise e
    if r.text:
        return r.json()
    else:
        return None

We can list users and their status:

In [3]:
api_request('users')

[{'admin': True,
  'last_activity': '2016-05-06T11:53:51.627000',
  'name': 'minrk',
  'pending': None,
  'server': '/user/minrk'},
 {'admin': False,
  'last_activity': '2016-05-06T11:57:13.329254',
  'name': 'takluyver',
  'pending': None,
  'server': None}]

We can also start user servers:

In [4]:
api_request('users/takluyver/server', method='post')
api_request('users')

[{'admin': True,
  'last_activity': '2016-05-06T11:53:51.627000',
  'name': 'minrk',
  'pending': None,
  'server': '/user/minrk'},
 {'admin': False,
  'last_activity': '2016-05-06T11:57:29.285044',
  'name': 'takluyver',
  'pending': None,
  'server': '/user/takluyver'}]

And stop them:

In [5]:
api_request('users/takluyver/server', method='delete')
api_request('users')

[{'admin': True,
  'last_activity': '2016-05-06T11:53:51.627000',
  'name': 'minrk',
  'pending': None,
  'server': '/user/minrk'},
 {'admin': False,
  'last_activity': '2016-05-06T11:57:30.145986',
  'name': 'takluyver',
  'pending': None,
  'server': None}]

We can also see the proxy routing table:

In [6]:
api_request('proxy')

{'/': {'last_activity': '2016-05-06T11:56:29.543Z',
  'target': 'http://127.0.0.1:8081'},
 '/user/minrk': {'last_activity': '2016-05-06T11:57:29.159Z',
  'target': 'http://127.0.0.1:45683',
  'user': 'minrk'}}